# Improved Question-Answering Search Engines with Transformers and Rerankers


![](https://i.imgur.com/7SXKckD.png)

Transfer Learning is the power of leveraging already trained models and tune \ adapt them to our own downstream tasks.

# QA Search Engine using Transformers

## Retrival and Re-ranking

In Semantic Search we have shown how to use SentenceTransformer to compute embeddings for queries, sentences, and paragraphs and how to use this for semantic search.

For complex search tasks, for example, for question answering retrieval, the search can significantly be improved by using Retrieve & Re-Rank.


## Retrieve & Re-Rank Pipeline

A pipeline for information retrieval / question answering retrieval that works well is the following. All components are provided and explained in this notebook:

![](https://i.imgur.com/yIXJRSo.png)


Given a search query, we first use a retrieval system that retrieves a large list of e.g. 100 possible hits which are potentially relevant for the query.
For the retrieval, we can use either lexical search, e.g. with ElasticSearch, or we can use dense retrieval with a bi-encoder. Simple Lexical searches can be based on TF-IDF, BM25 etc.


However, the retrieval system might retrieve documents that are not that relevant for the search query.
Hence, in a second stage, we use a re-ranker based on a cross-encoder that scores the relevancy of all candidates for the given search query.

The output will be a ranked list of hits we can present to the user.


## Retrieval: Bi-Encoder

For the retrieval of the candidate set, we can either use lexical search (e.g. ElasticSearch), or we can use a bi-encoder (semantic search) which is implemented in this repository.

Lexical search looks for literal matches of the query words in your document collection. It will not recognize synonyms, acronyms or spelling variations.

In contrast, semantic search (or dense retrieval) encodes the search query into vector space and retrieves the document embeddings that are close in vector space.

Bi-Encoders produce for a given sentence or document an embedding.


## Re-Ranker: Cross-Encoder

The retriever has to be efficient for large document collections with millions of entries. However, it might return irrelevant candidates.

A re-ranker based on a Cross-Encoder can substantially improve the final results for the user. The query and a possible document is passed simultaneously to transformer network, which then outputs a single score between 0 and 1 indicating how relevant the document is for the given query.

![](https://i.imgur.com/PFgkrcI.png)

The advantage of Cross-Encoders is the higher performance, as they perform attention across the query and the document.

Scoring thousands or millions of (query, document)-pairs would be rather slow. Hence, we use the retriever to create a set of e.g. 100 possible candidates which are then re-ranked by the Cross-Encoder.

First, you use an efficient Bi-Encoder to retrieve e.g. the top-100 most similar sentences for a query. Then, you use a Cross-Encoder to re-rank these 100 hits by computing the score for every (query, hit) combination.





## Retrieve & Re-Rank Search Engine over Simple Wikipedia

This examples demonstrates the Retrieve & Re-Rank Setup and allows to search over Simple Wikipedia.

You can input a query or a question. The script then uses semantic search to find relevant passages in Simple English Wikipedia

___[Created By: Dipanjan (DJ)](https://www.linkedin.com/in/dipanjans/)___

In [ ]:
!nvidia-smi

Tue Jun 18 11:45:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Install Dependencies

In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 7.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

### Load Transformer Models, Wikipedia Data and Generate Embeddings

For semantic search, we use `SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')` and retrieve 32 potentially relevant passages that answer the input query.

Next, we use a more powerful CrossEncoder `(cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2'))` that scores the query and all retrieved passages for their relevancy. The cross-encoder further boost the performance.

MS MARCO is a large scale information retrieval corpus that was created based on real user search queries using Bing search engine.

The provided models can be used for semantic search, i.e., given keywords / a search phrase / a question, the model will find passages that are relevant for the search query.

## Load Wikipedia Dataset

In [ ]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch


# As dataset, we use Simple English Wikipedia. Compared to the full English wikipedia, it has only
# about 170k articles. We split these articles into paragraphs and encode them with the bi-encoder

wikipedia_filepath = 'simplewiki-2020-11-01.jsonl.gz'

if not os.path.exists(wikipedia_filepath):
    util.http_get('http://sbert.net/datasets/simplewiki-2020-11-01.jsonl.gz', wikipedia_filepath)

passages = []
with gzip.open(wikipedia_filepath, 'rt', encoding='utf8') as fIn:
    for line in fIn:
        data = json.loads(line.strip())

        #Add all paragraphs
        #passages.extend(data['paragraphs'])

        #Only add the first paragraph
        passages.append(data['paragraphs'][0])

print("Passages:", len(passages))

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


  0%|          | 0.00/50.2M [00:00<?, ?B/s]

Passages: 169597


## Subset Dataset

In [ ]:
# We subset our data so we only use a subset of wikipedia to run things faster

passages = [passage for passage in passages for x in ['india', 'north pole', 'nlp',
                                                      'natural language processing', 'linguistics',
                                                      'machine learning', 'artificial intelligence',
                                                      'cheetah', 'animal', 'jaguar']
                                                    if x in passage.lower()]

## Look at sample documents

In [ ]:
len(passages)

5917

In [ ]:
passages[4]

'The Simputer is a small, cheap device, similar to personal digital assistant (palmtop). The unique feature of this device is a smart-card interface which allows a single Simputer to be used by many users. User\'s private datas might be kept on a cheap smart-card. This device was constructed by Indian company- Simputer Trust. The name stands for "Simple computer".'

## Load Transformer Models

In [ ]:
if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")


# We use the Bi-Encoder to encode all passages, so that we can use it with sematic search
bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
# The bi-encoder will retrieve 100 documents.
# We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

## Get Wikipedia Document Embeddings

In [ ]:
# We encode all passages into our vector space. This takes about few seconds (depends on your GPU speed)
corpus_embeddings = bi_encoder.encode(passages)

In [ ]:
passages[4]

'The Simputer is a small, cheap device, similar to personal digital assistant (palmtop). The unique feature of this device is a smart-card interface which allows a single Simputer to be used by many users. User\'s private datas might be kept on a cheap smart-card. This device was constructed by Indian company- Simputer Trust. The name stands for "Simple computer".'

In [ ]:
corpus_embeddings[4], corpus_embeddings[4].shape

(array([-9.37951803e-02, -4.31462713e-02, -2.95233093e-02, -1.15103293e-02,
         1.95731502e-02, -9.86015238e-03, -1.14418836e-02,  5.50647778e-03,
         4.94075976e-02,  9.13835224e-03,  6.34941086e-02,  6.97766244e-02,
        -4.67244349e-03, -3.26074548e-02, -1.02824513e-02, -9.35850143e-02,
         6.00409657e-02, -4.23400067e-02, -1.14094634e-02, -3.95792201e-02,
         2.55190227e-02, -3.53025682e-02, -4.68843915e-02,  2.80401446e-02,
        -1.89223082e-03, -1.17327198e-02,  4.57624253e-03, -9.29787382e-02,
         1.56899802e-02,  3.22136618e-02,  8.29601809e-02,  3.53492275e-02,
         1.48993134e-02,  8.06965008e-02, -1.37238661e-02, -4.12121341e-02,
         2.97504663e-02,  7.68969133e-02, -7.63456970e-02, -1.01581126e-01,
         5.85949644e-02, -9.55741554e-02,  1.25288814e-02,  5.11194244e-02,
        -5.84072024e-02,  3.80749144e-02, -4.47745472e-02, -6.78153411e-02,
        -7.63198137e-02, -6.73299655e-02, -3.75544690e-02,  3.97649035e-02,
         1.4

In [ ]:
corpus_embeddings.shape

(5917, 384)

## Try Search with a Sample Query

### New Query

In [ ]:
query = "What is the capital of India?"
query

'What is the capital of India?'

### Get Embedding for New Query

In [ ]:
query_embedding = bi_encoder.encode(query)
query_embedding.shape

(384,)

### Get Cosine Similarity Score of Document Emebddings compared to New Query Embedding

In [ ]:
cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
cos_scores

tensor([ 0.0209, -0.0524,  0.2248,  ...,  0.1950, -0.1026,  0.3683])

### Get Most Similar Document ID

In [ ]:
top_results = torch.topk(cos_scores, k=1)
idx = top_results.indices
idx

tensor([94])

### Get Most Similar Document

In [ ]:
passages[idx]

"Mumbai (previously known as Bombay until 1996) is a natural harbor on the west coast of India, and is the capital city of Maharashtra state. It is India's largest city, and one of the world's most populous cities. It is the financial capital of India. The city is the second most-populous in the world. It has approximately 13 million people. Along with the neighboring cities of Navi Mumbai and Thane, it forms the world's 4th largest urban agglomeration. They have around 19.1 million people."

## Alternate way of getting most similar document

In [ ]:
hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=1)
hits[0]

[{'corpus_id': 94, 'score': 0.5979241728782654}]

In [ ]:
hits[0][0]['corpus_id']

94

## Bi Encoder + ReRanker Cross Encoder Search

Re-ranker strategy is similar to what is used in RAG. It has been trained on pairs of documents and returns a score that corresponds to how similar the two documents are to each other. The cross encoder model has been fine-tuned on document pairs. Given a pair it outputs the similarity score.

### Get top K Similar documents from Bi-encoder and format input data for Reranker Cross-encoder

In [ ]:
# Get top 30 similar documents (hits) to the query
hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=30)
hits = hits[0]
# Format data for the reranker -> [query, similar_doc] for each of the top_k similar documents
reranker_inp = [[query, passages[hit['corpus_id']]] for hit in hits]
reranker_inp[:3] # look at the first 3 query inputs to the reranker cross encoder model

[['What is the capital of India?',
  "Mumbai (previously known as Bombay until 1996) is a natural harbor on the west coast of India, and is the capital city of Maharashtra state. It is India's largest city, and one of the world's most populous cities. It is the financial capital of India. The city is the second most-populous in the world. It has approximately 13 million people. Along with the neighboring cities of Navi Mumbai and Thane, it forms the world's 4th largest urban agglomeration. They have around 19.1 million people."],
 ['What is the capital of India?',
  "Kolkata (spelled Calcutta before 1 January 2001) is the capital city of the Indian state of West Bengal. It is the second largest city in India after Mumbai. It is on the east bank of the River Hooghly. When it is called Calcutta, it includes the suburbs. This makes it the third largest city of India. This also makes it the world's 8th largest metropolitan area as defined by the United Nations. Kolkata served as the capita

### Get Reranker score for every similar document

In [ ]:
reranker_scores = cross_encoder.predict(reranker_inp)
reranker_scores[:3] # look at relevance scores from reranker cross encoder

array([3.861031 , 3.4595218, 2.7084024], dtype=float32)

### Add Reranker score back to the hits dictionary

In [ ]:
hits[:3]

[{'corpus_id': 94, 'score': 0.5979241728782654},
 {'corpus_id': 789, 'score': 0.5937108993530273},
 {'corpus_id': 4586, 'score': 0.5878057479858398}]

In [ ]:
for id, hit in enumerate(hits):
    hit['reranker_score'] = reranker_scores[id]
hits[:3]

[{'corpus_id': 94, 'score': 0.5979241728782654, 'reranker_score': 3.861031},
 {'corpus_id': 789, 'score': 0.5937108993530273, 'reranker_score': 3.4595218},
 {'corpus_id': 4586, 'score': 0.5878057479858398, 'reranker_score': 2.7084024}]

### Show the top similar document to query based on both models

In [ ]:
print("Top Bi-Encoder Retrieval hit: ")
hit = sorted(hits, key=lambda x: x['score'], reverse=True)[0]
print(passages[hit['corpus_id']])

print("Top Reranker Retrieval hit: ")
hit = sorted(hits, key=lambda x: x['reranker_score'], reverse=True)[0]
print(passages[hit['corpus_id']])

Top Bi-Encoder Retrieval hit: 
Mumbai (previously known as Bombay until 1996) is a natural harbor on the west coast of India, and is the capital city of Maharashtra state. It is India's largest city, and one of the world's most populous cities. It is the financial capital of India. The city is the second most-populous in the world. It has approximately 13 million people. Along with the neighboring cities of Navi Mumbai and Thane, it forms the world's 4th largest urban agglomeration. They have around 19.1 million people.
Top Reranker Retrieval hit: 
New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of about 42.7 km. New Delhi has a population of about 9.4 Million people.


## Create a function to return the top similar document based on any query

In [ ]:
def search(query, top_k=30):
  # print the input question
  print("Input question:", query)

  ##### Bi-Encoder: Sematic Search #####
  # Encode the query using the bi-encoder and find potentially relevant passages
  query_embedding = bi_encoder.encode(query)
  hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=top_k)
  hits = hits[0]

  ##### Cross-Encoder: Re-Ranking #####
  # Now, score all retrieved passages with the reranker cross encoder
  reranker_inp = [[query, passages[hit['corpus_id']]] for hit in hits]
  reranker_scores = cross_encoder.predict(reranker_inp)

  # Store reranker cross encoder scores back into the hits variable
  for id, hit in enumerate(hits):
      hit['reranker_score'] = reranker_scores[id]

  # Output of top-1 hit from bi-encoder
  print("Top Bi-Encoder Retrieval hit: ")
  hit = sorted(hits, key=lambda x: x['score'], reverse=True)[0]
  print(passages[hit['corpus_id']])

  # Output of top-1 hit from re-ranker
  print("Top Reranker Retrieval hit: ")
  hit = sorted(hits, key=lambda x: x['reranker_score'], reverse=True)[0]
  print(passages[hit['corpus_id']])


## Try out the function

In [ ]:
search(query = "What is the capital of India?")

Input question: What is the capital of India?
Top Bi-Encoder Retrieval hit: 
Mumbai (previously known as Bombay until 1996) is a natural harbor on the west coast of India, and is the capital city of Maharashtra state. It is India's largest city, and one of the world's most populous cities. It is the financial capital of India. The city is the second most-populous in the world. It has approximately 13 million people. Along with the neighboring cities of Navi Mumbai and Thane, it forms the world's 4th largest urban agglomeration. They have around 19.1 million people.
Top Reranker Retrieval hit: 
New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of about 42.7 km. New Delhi has a population of about 9.4 Million people.


In [ ]:
search(query = "What is natural language processing?")

Input question: What is natural language processing?
Top Bi-Encoder Retrieval hit: 
Natural Language Processing (NLP) is a field in Artificial Intelligence, and is also related to linguistics. On a high level, the goal of NLP is to program computers to automatically understand human languages, and also to automatically write/speak in human languages. We say "Natural Language" to mean human language, and to indicate that we are not talking about computer (programming) languages.
Top Reranker Retrieval hit: 
Natural Language Processing (NLP) is a field in Artificial Intelligence, and is also related to linguistics. On a high level, the goal of NLP is to program computers to automatically understand human languages, and also to automatically write/speak in human languages. We say "Natural Language" to mean human language, and to indicate that we are not talking about computer (programming) languages.


In [ ]:
search(query = "What is language?")

Input question: What is language?
Top Bi-Encoder Retrieval hit: 
Philosophy of language is the study of how languages were created and are used. It is part of Linguistics. In continental philosophy, it is not treated as a subject by itself, but Ludwig Wittgenstein and other analytic philosophers placed particular stress on it.
Top Reranker Retrieval hit: 
Language is the normal way humans communicate. Only humans use language, though other animals communicate through other means.


In [ ]:
search(query = "What is coldest place on earth?")

Input question: What is coldest place on earth?
Top Bi-Encoder Retrieval hit: 
East Antarctica, also called Greater Antarctica, is the largest part (two-thirds) of the Antarctic continent. It is on the Indian Ocean side of the Transantarctic Mountains. It is the coldest, windiest, and driest part of Earth. East Antarctica holds the record as the coldest place on earth.
Top Reranker Retrieval hit: 
East Antarctica, also called Greater Antarctica, is the largest part (two-thirds) of the Antarctic continent. It is on the Indian Ocean side of the Transantarctic Mountains. It is the coldest, windiest, and driest part of Earth. East Antarctica holds the record as the coldest place on earth.


In [ ]:
search(query = "What is the animal which can run very fast?")

Input question: What is the animal which can run very fast?
Top Bi-Encoder Retrieval hit: 
Running is the way in which people or animals travel quickly on their feet. It is a method of travelling on land. It is different to walking in that both feet are regularly off the ground at the same time. Different terms are used to refer to running according to the speed: jogging is slow, and sprinting is running fast.
Top Reranker Retrieval hit: 
A cheetah ("Acinonyx jubatus") is a medium large cat which lives in Africa. It is the fastest land animal and can run up to 112 kilometers per hour for a short time. Most cheetahs live in the savannas of Africa. There are a few in Asia. Cheetahs are active during the day, and hunt in the early morning or late evening.
